# Calculate the $f$O$_2$ of a metal-silicate system from compositional data

For experiments that produced coexisting metallic alloy and silicate melt, the experimental $f$O$_2$ is calculated below using both Fe-FeO and Si-SiO$_2$ equilibrium:

$$ FeO_{silicate-melt} = Fe_{metal} + \frac{1}{2}O_2$$
$$ SiO_{2_{silicate-melt}} = Si_{metal} + O_2 $$


where the $f$O$_2$ of the experiment can be expressed relative to the $f$O$_2$ of the iron-wüstite (IW) buffer or Si-SiO$_2$ buffer, respectively, as:

$$ 2log(\alpha_{FeO}/\alpha_{Fe}) = 2log(X_{FeO} \centerdot \gamma_{FeO}/X_{Fe} \centerdot \gamma_{Fe}) = \Delta IW $$
$$ 2log(\alpha_{SiO_2}/\alpha_{Si}) = 2log(X_{SiO_2} \centerdot \gamma_{SiO_2} / X_{Si} \centerdot \gamma_{Si}) = \Delta SiSiO_2 $$

where $\alpha$, X, and $\gamma$ are the activity, mole fraction, and fugacity coefficient, respectively, of the subscripted component in the silicate melt or metal. "$\Delta$"  indicates the number of log units above or below the listed buffer. Experimental $f$O$_2$s are calculated using a non-ideal solution model for both metallic and silicate liquid. For each experiment, the activity coefficient for FeO in the silicate, $\gamma$FeO, is calculated with a parameterization based on Holzheid et al. (1997), where $\gamma$FeO is taken as a fixed value in the range 1.7–3 dependent only upon MgO content of the silicate melt: where MgO ≤20 wt%, $\gamma$FeO is set equal to 1.7; where MgO is >20 wt%, $\gamma$FeO is calculated as:

$$ \gamma_{FeO} = 1.7 + 0.1(MgO_{silicate, wt\%} - 20) $$

with a maximum allowable $\gamma$FeO value of 3.0. The activity of SiO$_2$ in the silicate melt is calculated using MELTS in the ENKI python framework (v. 1.1.0). The activity coefficients for Fe and Si in Fe-rich metal, $\gamma$Fe and $\gamma$Si, are calculated using the $\varepsilon$-approach (Wagner, 1952; Ma, 2001), which considers non-ideal interactions between components within the metal alloy. 

## 1. Import necessary python libraries

In [1]:
import fO2calculate as fc # code written for this work
from thermoengine import equilibrate # ENKI MELTS

/opt/anaconda3/lib/python3.9/site-packages/numdifftools/multicomplex.py:35: DeprecationWarning: `finfo.machar` is deprecated (NumPy 1.22)
  _TINY = np.finfo(float).machar.tiny


## 2. Read in your data file

You can create a single sample object or import an entire spreadsheet (xlsx or csv) with multiple samples. Here we import a spreadsheet containing data from this manuscript.

The BatchFile must have compositional information on each sample in rows. By default, the column used as the index column (corresponding to the name of each sample) is titled "Label". If no "Label" column is found, fo2calculate will choose the first column not containing compositional information as the index column. If an xlsx file is imported, the user can specify which sheet to import by passing the `sheet_name` argument. If no sheet name is given, fo2calculate will import the first sheet in the file.

Both silicate melt and metal compositions should be given together for each sample. Columns titled as oxides (e.g., "SiO2") are identified automatically as belonging to the silicate melt composition. Columns titled as elements (e.g., "Si") are identified as belonging to the metal composition. Note that subscripts, spaces, special characters, or other text such as units should not be used in column titles. Extra information about a sample, for example the temperature to use for calculations, can be given here if desired and have no requirements for how they are named. Any columns not used as the index column or as compositional information will be imported but not used by fO2calculate.

In [2]:
myfile = fc.BatchFile('MercurySmelting_Data/Compositions_for_Python.xlsx', sheet_name='All')

Next, we display the contents of the BatchFile to ensure they were imported properly.

In [3]:
myfile.get_data()

/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element S was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +


,Al2O3,CaO,Cr,Cr2O3,Fe,FeO,K2O,MgO,MnO,Na2O,P,S,Si,SiO2,Ti,TiO2,Temp_C,Total WT%,Which metal? Si-poor/Si-rich
Sample A (1),24.37,4.75,3.251301,0.12,84.633854,0.18,0.00,16.86,0.00,0.00,0.190076,0.000000,11.934774,51.11,0.000000,2.59,1340.0,99.96,0.0
Sample A (2),24.37,4.75,3.310400,0.12,84.594675,0.18,0.00,16.86,0.00,0.00,0.169508,0.000000,11.855619,51.11,0.069798,2.59,1340.0,100.29,0.0
Sample A (3),24.37,4.75,3.407496,0.12,84.606133,0.18,0.00,16.86,0.00,0.00,0.100220,0.000000,11.896172,51.11,0.000000,2.59,1340.0,99.78,0.0
Sample A (4),24.37,4.75,3.729910,0.12,81.714343,0.18,0.00,16.86,0.00,0.00,0.363894,0.000000,14.100879,51.11,0.090973,2.59,1340.0,98.93,0.0
Sample A (5),24.37,4.75,1.964736,0.12,85.239295,0.18,0.00,16.86,0.00,0.00,0.000000,0.000000,12.654912,51.11,0.141058,2.59,1340.0,99.25,0.0
Sample A (6),24.37,4.75,2.008275,0.12,85.074175,0.18,0.00,16.86,0.00,0.00,0.000000,0.000000,12.776264,51.11,0.141286,2.59,1340.0,99.09,0.0
Sample A (7),24.37,4.75,2.332421,0.12,84.301795,0.18,0.00,16.86,0.00,0.00,0.081128,0.000000,13.142683,51.11,0.131832,2.59,1340.0,98.61,0.0
Sample A (8),24.37,4.75,2.457943,0.12,84.748665,0.18,0.00,16.86,0.00,0.00,0.060441,0.000000,12.581847,51.11,0.151103,2.59,1340.0,99.27,0.0
Sample A (9),24.37,4.75,1.184930,0.12,89.457160,0.18,0.00,16.86,0.00,0.00,0.151914,0.000000,9.094592,51.11,0.121531,2.59,1340.0,98.74,0.0
Sample A (10),24.37,4.75,4.852541,0.12,87.680402,0.18,0.00,16.86,0.00,0.00,0.167329,0.000000,7.069651,51.11,0.219619,2.59,1340.0,95.62,0.0


## 3. Calculate the $f$O$_2$ in terms of the iron-wüstite buffer

Here we calculate the $\Delta$IW values for all measurements and display the results.

In [4]:
dIWs = myfile.calculate_dIW(temperature='Temp_C', pressure=1)
dIWs

[                    ] 2%  Working on sample Sample A (1)                            

/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element S was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +
/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element Cl was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +
/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element F was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +


[====================] 100%  Working on sample Sample O (9)                            


,Al2O3,CaO,Cr,Cr2O3,Fe,FeO,K2O,MgO,MnO,Na2O,...,Si,SiO2,Ti,TiO2,Temp_C,Total WT%,Which metal? Si-poor/Si-rich,dIW_Calculated,logfO2_dIW,Pressure_Modeled
Sample A (1),24.37,4.75,3.251301,0.12,84.633854,0.18,0.00,16.86,0.00,0.00,...,11.934774,51.11,0.000000,2.59,1340.0,99.96,0.0,-4.592162,-15.509828,1
Sample A (2),24.37,4.75,3.310400,0.12,84.594675,0.18,0.00,16.86,0.00,0.00,...,11.855619,51.11,0.069798,2.59,1340.0,100.29,0.0,-4.596586,-15.514253,1
Sample A (3),24.37,4.75,3.407496,0.12,84.606133,0.18,0.00,16.86,0.00,0.00,...,11.896172,51.11,0.000000,2.59,1340.0,99.78,0.0,-4.594302,-15.511969,1
Sample A (4),24.37,4.75,3.729910,0.12,81.714343,0.18,0.00,16.86,0.00,0.00,...,14.100879,51.11,0.090973,2.59,1340.0,98.93,0.0,-4.420202,-15.337869,1
Sample A (5),24.37,4.75,1.964736,0.12,85.239295,0.18,0.00,16.86,0.00,0.00,...,12.654912,51.11,0.141058,2.59,1340.0,99.25,0.0,-4.553620,-15.471286,1
Sample A (6),24.37,4.75,2.008275,0.12,85.074175,0.18,0.00,16.86,0.00,0.00,...,12.776264,51.11,0.141286,2.59,1340.0,99.09,0.0,-4.544111,-15.461778,1
Sample A (7),24.37,4.75,2.332421,0.12,84.301795,0.18,0.00,16.86,0.00,0.00,...,13.142683,51.11,0.131832,2.59,1340.0,98.61,0.0,-4.512190,-15.429857,1
Sample A (8),24.37,4.75,2.457943,0.12,84.748665,0.18,0.00,16.86,0.00,0.00,...,12.581847,51.11,0.151103,2.59,1340.0,99.27,0.0,-4.553184,-15.470851,1
Sample A (9),24.37,4.75,1.184930,0.12,89.457160,0.18,0.00,16.86,0.00,0.00,...,9.094592,51.11,0.121531,2.59,1340.0,98.74,0.0,-4.797845,-15.715512,1
Sample A (10),24.37,4.75,4.852541,0.12,87.680402,0.18,0.00,16.86,0.00,0.00,...,7.069651,51.11,0.219619,2.59,1340.0,95.62,0.0,-4.869998,-15.787665,1


Here we save the results to an excel file.

In [5]:
myfile.save_excel(filename='dIW.xlsx', calculations=[dIWs])

Saved dIW.xlsx


## 4. Calculate the $f$O$_2$ in terms of the silicon-silica buffer

### 4.1 First, instantiate a MELTS class and calculate the activity of SiO$_2$ in the melt for each sample

In [6]:
# -------------- MELTS preamble --------------- #
# instantiate thermoengine equilibrate MELTS instance
melts = equilibrate.MELTSmodel('1.1.0')

# Suppress phases not required in the melts simulation
phases = melts.get_phase_names()
for phase in phases:
    melts.set_phase_inclusion_status({phase: False})
melts.set_phase_inclusion_status({'Fluid': True, 'Liquid': True})
# --------------------------------------------- #

In [7]:
# extract only silicate compositions from the dataset
silicate_compositions = [myfile.get_sample_composition(samplename=i, how='silicate') for i in myfile.get_data().index]

# extract the temperature corresponding to each sample
temperatures = [row['Temp_C'] for index, row in myfile.get_data().iterrows()]

/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element S was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +


In [8]:
# calculate the silica activity using ENKI MELTS and save those values to a list
aSiO2_list = []
for i, comp in enumerate(silicate_compositions):
    del comp['S']
    melts.set_bulk_composition(comp)
    output = melts.equilibrate_tp(temperatures[i], 0.1, initialize=True)
    (status, temperature, pressureMPa, xmlout) = output[0]
    activities = melts.get_thermo_properties_of_phase_components(xmlout, phase_name='Liquid', mode='activity')
    aSiO2_list.append(activities['SiO2'])

In [9]:
# add silica activities to our BatchFile
myfile.data['aSiO2'] = aSiO2_list

Here we print the BatchFile to ensure the data was calculated and written correctly

In [10]:
myfile.get_data()

,Al2O3,CaO,Cr,Cr2O3,Fe,FeO,K2O,MgO,MnO,Na2O,P,S,Si,SiO2,Ti,TiO2,Temp_C,Total WT%,Which metal? Si-poor/Si-rich,aSiO2
Sample A (1),24.37,4.75,3.251301,0.12,84.633854,0.18,0.00,16.86,0.00,0.00,0.190076,0.000000,11.934774,51.11,0.000000,2.59,1340.0,99.96,0.0,0.475599
Sample A (2),24.37,4.75,3.310400,0.12,84.594675,0.18,0.00,16.86,0.00,0.00,0.169508,0.000000,11.855619,51.11,0.069798,2.59,1340.0,100.29,0.0,0.475599
Sample A (3),24.37,4.75,3.407496,0.12,84.606133,0.18,0.00,16.86,0.00,0.00,0.100220,0.000000,11.896172,51.11,0.000000,2.59,1340.0,99.78,0.0,0.475599
Sample A (4),24.37,4.75,3.729910,0.12,81.714343,0.18,0.00,16.86,0.00,0.00,0.363894,0.000000,14.100879,51.11,0.090973,2.59,1340.0,98.93,0.0,0.475599
Sample A (5),24.37,4.75,1.964736,0.12,85.239295,0.18,0.00,16.86,0.00,0.00,0.000000,0.000000,12.654912,51.11,0.141058,2.59,1340.0,99.25,0.0,0.475599
Sample A (6),24.37,4.75,2.008275,0.12,85.074175,0.18,0.00,16.86,0.00,0.00,0.000000,0.000000,12.776264,51.11,0.141286,2.59,1340.0,99.09,0.0,0.475599
Sample A (7),24.37,4.75,2.332421,0.12,84.301795,0.18,0.00,16.86,0.00,0.00,0.081128,0.000000,13.142683,51.11,0.131832,2.59,1340.0,98.61,0.0,0.475599
Sample A (8),24.37,4.75,2.457943,0.12,84.748665,0.18,0.00,16.86,0.00,0.00,0.060441,0.000000,12.581847,51.11,0.151103,2.59,1340.0,99.27,0.0,0.475599
Sample A (9),24.37,4.75,1.184930,0.12,89.457160,0.18,0.00,16.86,0.00,0.00,0.151914,0.000000,9.094592,51.11,0.121531,2.59,1340.0,98.74,0.0,0.475599
Sample A (10),24.37,4.75,4.852541,0.12,87.680402,0.18,0.00,16.86,0.00,0.00,0.167329,0.000000,7.069651,51.11,0.219619,2.59,1340.0,95.62,0.0,0.475599


### 4.2 Next, use the fO2calculate module to calculate $\Delta$SiSiO$_2$ for all samples

Here we calculate $\Delta$SiSiO$_2$ for all measurements and print the results.

In [11]:
dSiSiO2s = myfile.calculate_dSiSiO2(temperature='Temp_C', aSiO2='aSiO2', pressure=1)
dSiSiO2s

[                    ] 3%  Working on sample Sample A (2)                            

/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element S was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +
/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element Cl was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +
/opt/anaconda3/lib/python3.9/site-packages/fO2calculate/sample_class.py:75: UserWarning: Element F was passed as part of sample composition. This will be assumed to be part of the metal composition.
  w.warn("Element " + str(vol) + " was passed as part of sample composition. This" +


[====================] 100%  Working on sample Sample O (9)                            


,Al2O3,CaO,Cr,Cr2O3,Fe,FeO,K2O,MgO,MnO,Na2O,...,Ti,TiO2,Temp_C,Total WT%,Which metal? Si-poor/Si-rich,aSiO2,dSiSiO2_Calculated,logfO2_dSiSiO2,dIW_SiSiO2,Pressure_Modeled
Sample A (1),24.37,4.75,3.251301,0.12,84.633854,0.18,0.00,16.86,0.00,0.00,...,0.000000,2.59,1340.0,99.96,0.0,0.475599,4.754054,-15.410849,-4.493183,1
Sample A (2),24.37,4.75,3.310400,0.12,84.594675,0.18,0.00,16.86,0.00,0.00,...,0.069798,2.59,1340.0,100.29,0.0,0.475599,4.774649,-15.390255,-4.472588,1
Sample A (3),24.37,4.75,3.407496,0.12,84.606133,0.18,0.00,16.86,0.00,0.00,...,0.000000,2.59,1340.0,99.78,0.0,0.475599,4.762356,-15.402547,-4.484881,1
Sample A (4),24.37,4.75,3.729910,0.12,81.714343,0.18,0.00,16.86,0.00,0.00,...,0.090973,2.59,1340.0,98.93,0.0,0.475599,4.209257,-15.955646,-5.037979,1
Sample A (5),24.37,4.75,1.964736,0.12,85.239295,0.18,0.00,16.86,0.00,0.00,...,0.141058,2.59,1340.0,99.25,0.0,0.475599,4.558737,-15.606167,-4.688500,1
Sample A (6),24.37,4.75,2.008275,0.12,85.074175,0.18,0.00,16.86,0.00,0.00,...,0.141286,2.59,1340.0,99.09,0.0,0.475599,4.527638,-15.637265,-4.719599,1
Sample A (7),24.37,4.75,2.332421,0.12,84.301795,0.18,0.00,16.86,0.00,0.00,...,0.131832,2.59,1340.0,98.61,0.0,0.475599,4.436766,-15.728137,-4.810470,1
Sample A (8),24.37,4.75,2.457943,0.12,84.748665,0.18,0.00,16.86,0.00,0.00,...,0.151103,2.59,1340.0,99.27,0.0,0.475599,4.580415,-15.584489,-4.666822,1
Sample A (9),24.37,4.75,1.184930,0.12,89.457160,0.18,0.00,16.86,0.00,0.00,...,0.121531,2.59,1340.0,98.74,0.0,0.475599,5.542834,-14.622069,-3.704402,1
Sample A (10),24.37,4.75,4.852541,0.12,87.680402,0.18,0.00,16.86,0.00,0.00,...,0.219619,2.59,1340.0,95.62,0.0,0.475599,6.184085,-13.980818,-3.063151,1


Finally, we save all $\Delta$SiSiO$_2$ values to an excel spreadsheet.

In [12]:
myfile.save_excel(filename='dSiSiO2.xlsx', calculations=[dSiSiO2s])

Saved dSiSiO2.xlsx
